In [25]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [26]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.3751043  -0.7733845   0.592771   ...  0.57923514  0.3268347
  -0.65089667]
 [-0.09361713 -0.18191509 -0.1923083  ... -0.0316581   0.30412516
  -0.26793626]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [27]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [28]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [29]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [30]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [31]:
model.encode(df.loc[0, '유저'])

array([-4.80606586e-01, -2.94868886e-01,  4.37900156e-01, -6.40137494e-01,
        3.28669585e-02, -3.42647582e-01, -5.47482856e-02,  1.73054785e-02,
       -4.08221215e-01, -5.06034017e-01, -1.68733373e-01, -3.98677021e-01,
       -1.24776915e-01, -9.71541777e-02, -1.65286496e-01,  5.72590437e-03,
        6.13687076e-02, -1.91311821e-01,  2.53917634e-01, -5.85019410e-01,
       -2.84426212e-01, -2.32035428e-01, -3.27080816e-01,  6.72979131e-02,
       -1.65434813e-05, -4.72336292e-01, -3.60022098e-01,  2.91879833e-01,
       -6.63861394e-01, -3.10574710e-01,  5.79525113e-01, -3.11722755e-01,
        1.47698680e-02, -2.12172553e-01,  2.22058401e-01, -1.73828974e-01,
       -3.78458411e-01, -4.20397997e-01, -2.38218948e-01,  6.38705269e-02,
       -1.15304455e-01, -2.44564041e-01, -5.00228465e-01,  1.68355629e-01,
       -6.58360183e-01, -8.91942024e-01, -6.26956880e-01, -3.21965188e-01,
       -7.05358028e-01,  3.71447146e-01, -5.45803607e-01,  7.76301548e-02,
        1.09864287e-01,  

# 유저 대화내용 인코딩

In [32]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806066, -0.2948689, 0.43790016, -0.640137..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.156157, -0.14506243, 0.29490376, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652001, -0.08126799, 1.0945563, 0.1057919..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679063, 0.46520722, 0.52850705, -0.507604..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277739, -0.37413907, 0.040532183, -0.862..."


In [33]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [34]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [35]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806066, -0.2948689, 0.43790016, -0.640137...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.156157, -0.14506243, 0.29490376, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652001, -0.08126799, 1.0945563, 0.1057919...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679063, 0.46520722, 0.52850705, -0.507604...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277739, -0.37413907, 0.040532183, -0.862...",0.315118


In [36]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

구분 일반
유사한 질문 머리가 너무 아프네
챗봇 답변 시원한 바람 쉬고 약먹기!
유사도 0.9030822515487671
